<a href="https://colab.research.google.com/github/ashishkhareiitr/projects/blob/master/pose_stick.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install pafy
!pip install youtube_dl

import sys
import PIL.Image
import cv2
import os
from sys import platform
import re
from os import listdir
import ntpath #for retrieving filename
import pafy
import youtube_dl
import pandas as pd

In [28]:
from google.colab import files
uploaded = files.upload()



Saving stickpose.zip to stickpose (1).zip


In [0]:
import zipfile
import io
data = zipfile.ZipFile(io.BytesIO(uploaded['stickpose.zip']), 'r')
data.extractall()

In [34]:

_nsre = re.compile('([0-9]+)')
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(_nsre, s)]     


def loadImages(path):
    # return array of images
    
    imagesList = listdir(path)
    imagesList.sort(key=natural_sort_key)
    #imagesList = imagesList.sort()     
    loadedImages = []
    for image in imagesList:
      if image is not None:
        img = PIL.Image.open(path + image)
        loadedImages.append(img)
      else:
        print('not loaded')
    
    
    
    return loadedImages


# Specify the paths for the 2 files
#protoFile = "A://ISB//Capstone//PoseRetreival//OpenCVPoseEsti//pose_deploy_linevec.prototxt"
protoFile = "pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "pose_iter_440000.caffemodel"
 
path_loadImages =  "stickpose/"
imgs = loadImages(path_loadImages)


#path_poseImages = r"A://ISB//Capstone//PoseRetreival//OpenCVPoseEsti//poseImages//"

# Read the network into Memory
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

## GET FRAMES
#pathOut = r"A:/ISB/Capstone/PoseRetreival/FramePerSecond/"
#url = 'https://www.youtube.com/watch?v=AbPufvvYiSw'
#url='https://www.youtube.com/watch?v=SCr5mEl9qbQ'
#vPafy = pafy.new(url)
#play = vPafy.getbest(preftype="webm")

#Get from the video from local folder
vidcap = cv2.VideoCapture("yoga3.mp4")


#Get from the video from youtube
#vidcap = cv2.VideoCapture(play.url)


count = 0
success = True
'''
while success:
    success,image = vidcap.read()
    print('read a new frame:',success)
    if count%90 == 0 :
     #    cv2.imshow('test',image)
     #    cv2.imwrite(pathOut + 'frame%d.jpg'%count,image)
     #    #cv2.waitKey(1000)
     #    #cv2.destroyAllWindows()
    #count+=1
'''    


    
        
for i in range(1,len(imgs)): 
        frame=cv2.imread(imgs[i-1].filename)
        
        # Read image
        #frame = cv2.imread("A://ISB//Capstone//PoseRetreival//OpenCVPoseEsti//Image//Image3.jpg")
        
        #frame = image
        ###########Resize the image##################
         
        '''
        #basewidth = 1280
        #img = PIL.Image.open("A://ISB//Capstone//PoseRetreival//OpenCVPoseEsti//Image//Image3.jpg")
        width, height = img.size
        #print(width,height)
         
        wpercent = (basewidth/float(img.size[0]))
        hsize = int((float(img.size[1])*float(wpercent)))
        img = img.resize((basewidth,hsize), PIL.Image.ANTIALIAS)
        '''
        #width, height = frame.size
        height, width, channels = frame.shape
        print(width,height)
        #img.save('sompic.jpg') 
        if width <600:
            x=width
            y= 600 -x
            width = width +x
            height =  height +int((height/x)*y)
            frame = cv2.resize(frame, (width, height)) 
       
        ################################################
        # Read image
        #frame = cv2.imread(imgs[i-1].filename) #cv2.imread("sompic.jpg")
         
        # Specify the input image dimensions
        inWidth = width +0
        inHeight = height + int((height/width)*0)
        #print(inWidth,inHeight)
        # Prepare the frame to be fed to the network
        inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight), (0, 0, 0), swapRB=False, crop=False)
         
        # Set the prepared object as the input blob of the network
        net.setInput(inpBlob)
        
        p=((0,1),(2,5),(3,6),(4,7),(8,11),(9,12),(10,13),(1,8),(1,11))        
        output = net.forward()
        
        H = output.shape[2]
        W = output.shape[3]
        # Empty list to store the detected keypoints
        points = {}
        for i in range(0,13):
            # confidence map of corresponding body's part.
            probMap = output[0, i, :, :]
        
            # Find global maxima of the probMap.
            minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
             
            # Scale the point to fit on the original image
            #x = (frameWidth * point[0]) / W
            #y = (frameHeight * point[1]) / H
            x = (inWidth * point[0]) / W
            y = (inHeight * point[1]) / H
            if prob > 0.01 : 
                cv2.circle(frame, (int(x), int(y)), 15, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
                cv2.putText(frame, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 3, lineType=cv2.LINE_AA)
         
                # Add the point to the list if the probability is greater than the threshold
                points.update({i:(int(x), int(y))})
                #print(prob)
            else :
                #points.update(None)
                 print(points)
        
        for pair in p:
            partA = pair[0]
            partB = pair[1]
            print(partA,partB)
            #if points[partA] and points[partB]:
            if (partA in points) and (partB in points):
                cv2.line(frame, points[partA], points[partB], (0, 255, 0), 3)    
        
        cv2.imwrite(os.path.join(os.getcwd(),"stickpose{:d}.jpg".format(count)),frame )         
        #cv2.imshow("Output-Keypoints",frame)
        #cv2.waitKey(5000)
        #cv2.destroyAllWindows()
        count+=1

474 327
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
480 480
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
500 500
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
1000 667
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
1080 1108
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
480 480
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
960 720
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
500 333
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
640 640
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
720 810
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
570 310
{0: (310, 55), 1: (350, 111), 2: (406, 103), 3: (223, 190), 4: (167, 270), 5: (310, 119), 6: (223, 214), 7: (167, 270), 8: (374, 238), 9: (231, 87)}
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
500 500
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
400 300
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
768 1024
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
400 500
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
450 360
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
640 360
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
1280 720
0 1
2

In [8]:
"""
Object detection
================
 
Shape dataset
"""
 
import os
 
import keras_rcnn
import keras_rcnn.datasets.shape
import keras_rcnn.preprocessing
import keras_rcnn.utils
import keras_rcnn.models
import numpy
import keras
 
import matplotlib
import matplotlib.backends.backend_pdf
 
import pickle
 
print(os.getcwd())
 
 
def main():
    training_dictionary, test_dictionary = keras_rcnn.datasets.shape.load_data()
 
    categories = {"circle": 1, "rectangle": 2, "triangle": 3}
 
    generator = keras_rcnn.preprocessing.ObjectDetectionGenerator()
 
    generator = generator.flow_from_dictionary(
        dictionary=training_dictionary,
        categories=categories,
        target_size=(224, 224)
    )
 
    validation_data = keras_rcnn.preprocessing.ObjectDetectionGenerator()
 
    validation_data = validation_data.flow_from_dictionary(
        dictionary=test_dictionary,
        categories=categories,
        target_size=(224, 224),
        shuffle=False
    )
 
    target, _ = generator.next()
 
    # Plotting expected predictions
    target_bounding_boxes, target_categories, target_images, target_masks, target_metadata = target
    target_images = numpy.squeeze(target_images)
    target_bounding_boxes = numpy.squeeze(target_bounding_boxes)
    target_categories = numpy.argmax(target_categories, -1)
    target_categories = numpy.squeeze(target_categories)
    keras_rcnn.utils.show_bounding_boxes(target_images, target_bounding_boxes, target_categories)
    matplotlib.pyplot.show()
    matplotlib.pyplot.gcf().clear()
 
    model = keras_rcnn.models.RCNN((224, 224, 3), ["circle", "rectangle", "triangle"])
 
    optimizer = keras.optimizers.Adam()
    model.compile(optimizer)
 
    model.fit_generator(
        epochs=100,
        generator=generator,
        validation_data=validation_data
    )
 
    # TODO: save model
    # model.save('rcnn_quickstart_1_epoch_model.h5')
 
    predictions = model.predict_generator(validation_data)
 
    pickle.dump(predictions, open('rcnn_quickstart_100_epochs_model_predictions.p', 'wb'))
    predictions = pickle.load(open('rcnn_quickstart_100_epochs_model_predictions.p', 'rb'))
 
    # Example prediction on validation data
    (target_bounding_boxes, target_categories, target_images, target_masks, _), _ = validation_data[60]
    target_images = numpy.squeeze(target_images)
    predicted_bad = predictions[0][60]
    predicted_bad = numpy.squeeze(predicted_bad)
    predicted_bad_category = predictions[1][60]
    predicted_bad_category = numpy.squeeze(predicted_bad_category)
    predicted_bad_category_idx = numpy.argmax(predicted_bad_category, axis=1)
    keras_rcnn.utils.show_bounding_boxes(target_images, predicted_bad, predicted_bad_category_idx)
    matplotlib.pyplot.show()
    matplotlib.pyplot.gcf().clear()
 
    # Plotting perfect predictions first
    pdf = matplotlib.backends.backend_pdf.PdfPages("rcnn_quickstart_100_epochs_model_expected_plot.pdf")
    for i in range(len(validation_data)):
        (target_bounding_boxes, target_categories, target_images, target_masks, _), _ = validation_data[i]
        target_bounding_boxes = numpy.squeeze(target_bounding_boxes)
        target_images = numpy.squeeze(target_images)
        target_categories = numpy.argmax(target_categories, -1)
        target_categories = numpy.squeeze(target_categories)
        keras_rcnn.utils.show_bounding_boxes(target_images, target_bounding_boxes, target_categories)
        pdf.savefig()
        matplotlib.pyplot.gcf().clear()
    pdf.close()
 
    pdf = matplotlib.backends.backend_pdf.PdfPages("rcnn_quickstart_100_epochs_model_predictions_plot.pdf")
    for i in range(len(validation_data)):
        (target_bounding_boxes, target_categories, target_images, target_masks, _), _ = validation_data[i]
        target_images = numpy.squeeze(target_images)
        predicted_bad = predictions[0][i]
        predicted_bad = numpy.squeeze(predicted_bad)
        predicted_bad_category = predictions[1][i]
        predicted_bad_category = numpy.squeeze(predicted_bad_category)
        predicted_bad_category_idx = numpy.argmax(predicted_bad_category, axis=1)
        keras_rcnn.utils.show_bounding_boxes(target_images, predicted_bad, predicted_bad_category_idx)
        pdf.savefig()
        matplotlib.pyplot.gcf().clear()
    pdf.close()
 
 
if __name__ == '__main__':
    main()

ModuleNotFoundError: ignored

In [0]:
import os
 

import keras
 
import matplotlib
import matplotlib.backends.backend_pdf
 
import pickle

import keras_rcnn


In [10]:
!pip install keras_rcnn.datasets.shape



  Could not find a version that satisfies the requirement keras_rcnn.datasets.shape (from versions: )
No matching distribution found for keras_rcnn.datasets.shape
